In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd
import sys
import hardware_control.wx_programs as wx
import hardware_control.bnc as bnc
from classes.generator import *
from hardware_control.hardware_config import *
from experiment_configuration.values import *
from classes.qubit_class import *
import daq.daq_programs_homo as daq
import seaborn as sns
import standard_sequences.pinopi as pnp
import analysis.plotting as plotting
from skopt import gp_minimize
import traceback
import time
import csv
import os
from skopt.space import Categorical

wx_addr = wx.get_wx_address()
main_directory = r"C:\Users\quantum1\Documents"
save_dir = rf"{main_directory}\Python Scripts\Important Blue Fridge Python Files\New\nonlinear_QM\data"
target_bnc_address_6 = "USB0::0x03EB::0xAFFF::411-433500000-0753::INSTR"
bnc.set_bnc_output(
    general_vals_dict["qubit_bnc"], power_dBm=13, bnc_addr=target_bnc_address_6
)
bnc.set_bnc_output(
    readout_dict["RO_LO"],
    power_dBm=readout_dict["RO_LO_pwr"],
    bnc_addr=bnc_address["target_bnc_black"],
)
bnc.set_bnc_output(
    general_vals_dict["TWPA_freq"],
   -2.5,
    bnc_addr=bnc_address["big_agilent"],
)

ModuleNotFoundError: No module named 'usbtmc'

In [ ]:
q1 = Qubit(q1_dict, readout_dict)
q2 = Qubit(q2_dict, readout_dict)
general_vals_dict["wx_offs"] = [0, 0, -0.08, 0]
readout = Readout(readout_dict)
print(f"{q1}\n{q2}")

In [ ]:
def pi_nopipi_twpa_loss(twpa_freq, twpa_power, reps=10000):
    """
    Loss function to optimize TWPA frequency and power by maximizing SNR.
    Computes SNR from raw IQ data using sample mean and std, no fitting.

    Parameters:
        twpa_freq (float): Frequency of the TWPA (Hz)
        twpa_power (float): Power into the TWPA (dBm)
        reps (int): Number of repetitions per pattern (default: 10,000)

    Returns:
        float: Negative SNR (loss to minimize)
    """
    # === [1] Set TWPA hardware parameters ===
    bnc.set_bnc_output(
        twpa_freq,
        twpa_power,
        bnc_addr=bnc_address["big_agilent"],
    )

    # === [2] Prepare measurement patterns ===
    pnp.pi_nopi_ge(0, 0, q2, q1, general_vals_dict)
    pnp.pi_nopi_ge(1, 1, q2, q1, general_vals_dict)
    wx.wx_set_and_amplitude_and_offset(
        amp=general_vals_dict["wx_amps"], offset=general_vals_dict["wx_offs"]
    )

    # === [3] Run the measurement ===
    values = daq.run_daq_het_2q(q1, q2, num_patterns=3, num_records_per_pattern=reps)
    readout_vs_pats = values.rec_readout_vs_pats_2

    # === [4] Extract IQ Data ===
    I_g = readout_vs_pats[0, :, 0]
    I_e = readout_vs_pats[0, :, 1]
    Q_g = readout_vs_pats[1, :, 0]
    Q_e = readout_vs_pats[1, :, 1]

    # === [5] Compute Means and Standard Deviations ===
    mu_I_g, mu_I_e = np.mean(I_g), np.mean(I_e)
    mu_Q_g, mu_Q_e = np.mean(Q_g), np.mean(Q_e)
    std_I_g, std_I_e = np.std(I_g), np.std(I_e)
    std_Q_g, std_Q_e = np.std(Q_g), np.std(Q_e)

    # === [6] Optional: Handle NaNs or failed measurements ===
    if np.any(np.isnan([mu_I_g, mu_I_e, mu_Q_g, mu_Q_e, std_I_g, std_I_e, std_Q_g, std_Q_e])):
        print("Warning: NaN detected in IQ data. Returning high loss.")
        return 1e6  # Penalize this config in optimizer

    # === [7] Compute SNR ===
    signal = np.sqrt((mu_I_g - mu_I_e)**2 + (mu_Q_g - mu_Q_e)**2)
    noise = (np.abs(std_I_g) + np.abs(std_I_e) + np.abs(std_Q_g) + np.abs(std_Q_e)) / 4
    SNR = signal / noise

    # === [8] Optional: Log parameters and SNR ===
    print(f"TWPA freq: {twpa_freq/1e9:.3f} GHz | power: {twpa_power:.1f} dBm | SNR: {SNR:.2f}")

    # === [9] Return negative SNR as loss ===
    return -SNR


NameError: name 'pnp' is not defined

In [ ]:
# Create or append to results CSV
CSV_PATH = "twpa_snr_log.csv"
if not os.path.exists(CSV_PATH):
    with open(CSV_PATH, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["TWPA Frequency (Hz)", "TWPA Power (dBm)", "SNR", "Loss"])

# Safe wrapper around your loss function with retry
def safe_loss_wrapper(params, reps=5000, max_retries=10, retry_wait=5):
    freq, power = params
    for attempt in range(max_retries):
        try:
            loss = pi_nopipi_twpa_loss(freq, power, reps=reps)
            SNR = -loss  # Since loss = -SNR
            # Log to CSV
            with open(CSV_PATH, "a", newline="") as f:
                writer = csv.writer(f)
                writer.writerow([freq, power, SNR, loss])
            return loss
        except Exception as e:
            print(f"Error during evaluation at freq={freq}, power={power}")
            traceback.print_exc()
            if attempt < max_retries - 1:
                print(f"Retrying in {retry_wait} seconds...")
                time.sleep(retry_wait)
            else:
                print("Max retries reached. Returning large loss.")
                return 1e3  # Penalize this setting



# Define resolution-aligned discrete grids
freq_grid = np.round(np.arange(3.0, 9.0 + 1e-9, 0.1), 3)
power_grid = np.round(np.arange(-10, 10 + 1e-9, 0.1), 2)

# Use Categorical to create discrete search space
param_space = [
    Categorical(freq_grid.tolist(), name='freq'),
    Categorical(power_grid.tolist(), name='power'),
]

# Run optimizer with the same safe wrapper and logging
result = gp_minimize(
    func=safe_loss_wrapper,
    dimensions=param_space,
    n_calls=30,
    n_initial_points=5,
    acq_func="EI",
    random_state=42,
)

In [ ]:

print(f"\nOptimization complete. Best SNR: {-result.fun:.2f}")
print(f"Best TWPA freq: {result.x[0]} GHz | power: {result.x[1]} dBm")